# **Case WGG Digital Solutions**

## **Problema de negócios**

O case técnico apresentado pela WGG Digital Solutions envolve uma base de dados que contém informações críticas para a tomada de decisão em um contexto empresarial. Os dados, registrados em um arquivo Excel denominado 'Forecast Value', incluem percentuais de atingimento de metas de venda, organizados por país e distribuídos ao longo de um período de tempo específico, de fevereiro de 2022 a dezembro de 2023.

Cada registro na base de dados representa o percentual de uma meta de vendas alcançada por um país em um determinado mês. Por exemplo, um registro pode indicar que a meta para a Colômbia e o Equador em fevereiro de 2022 foi de 58%.

O problema de negócio a ser resolvido envolve a previsão do percentual de atingimento de metas para janeiro de 2024, com base nos dados históricos disponíveis. A capacidade de prever esse valor é importante para o planejamento estratégico, alocação de recursos e definição de metas futuras, permitindo que a empresa tome decisões proativas para otimizar suas operações e alcançar seus objetivos de vendas.

Este desafio requer a aplicação de técnicas de análise de dados e modelagem preditiva para fornecer uma estimativa precisa, auxiliando a empresa a antecipar tendências de desempenho e a agir de forma eficaz no mercado.

Neste Notebook, vamos explorar e comparar o desempenho de diferentes modelos preditivos, incluindo Redes Neurais Recorrentes (RNN), Árvore de Decisão (DT), Floresta Aleatória (RF) e Regressão Linear (RL). Para cada modelo, calcularemos o Root Mean Squared Error (RMSE), que nos permitirá avaliar e selecionar o modelo mais eficaz para este case.

## Instalando as Bibliotecas necessarias

In [ ]:
pip install tensorflow

## Importando as Bibliotecas

In [ ]:
# Bibliotecas para manipulação de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização
import matplotlib.pyplot as plt

# Bibliotecas para aprendizado de máquina
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Ignorar avisos
import warnings
warnings.filterwarnings("ignore")

## Extraindo os Dados

In [3]:
df = pd.read_excel('/content/Forecast Value (3) (1).xlsx')
df.head()

,Month/Year,Value,Country
0,2022-02-01,0.58,Colombia/Ecuador
1,2022-03-04,0.61,Colombia/Ecuador
2,2022-04-04,0.65,Colombia/Ecuador
3,2022-05-05,0.39,Colombia/Ecuador
4,2022-06-05,0.53,Colombia/Ecuador


Realizaremos a separação de um DataFrame principal,* df*, em três DataFrames menores, cada um contendo os dados específicos de um dos países permitindo tratar os dados separadamente e criar modelos espeficicos para cada pais de forma organizada.

In [4]:
# Criar DataFrames separados para cada país
df_brasil = df[df['Country'] == 'Brasil'].reset_index(drop=True)
df_colombia_ecuador = df[df['Country'] == 'Colombia/Ecuador'].reset_index(drop=True)
df_peru = df[df['Country'] == 'Peru'].reset_index(drop=True)

# Exibir os DataFrames criados
print("DataFrame Brasil:\n", df_brasil.head())
print("\nDataFrame Colombia/Ecuador:\n", df_colombia_ecuador.head())
print("\nDataFrame Peru:\n", df_peru.head())

DataFrame Brasil:
   Month/Year  Value Country
0 2022-02-01   0.55  Brasil
1 2022-03-04   0.69  Brasil
2 2022-04-04   0.70  Brasil
3 2022-05-05   0.70  Brasil
4 2022-06-05   0.66  Brasil

DataFrame Colombia/Ecuador:
   Month/Year  Value           Country
0 2022-02-01   0.58  Colombia/Ecuador
1 2022-03-04   0.61  Colombia/Ecuador
2 2022-04-04   0.65  Colombia/Ecuador
3 2022-05-05   0.39  Colombia/Ecuador
4 2022-06-05   0.53  Colombia/Ecuador

DataFrame Peru:
   Month/Year  Value Country
0 2022-02-01   0.29    Peru
1 2022-03-04   0.35    Peru
2 2022-04-04   0.61    Peru
3 2022-05-05   0.72    Peru
4 2022-06-05   0.68    Peru


Para ficar ainda mais organizado para a criação de diversos modelos, de diversos paises, iremos dividir cada dataset em treino e teste para que possamos analisar cada pais de forma independente.

In [5]:
# Cópia para Redes Neurais Recorrentes (RNN), Árvore de Decisão (DT), Floresta Aleatória (RF) e  Regressão Linear (RL)

# Para Brasil
df_brasil_RNN, df_brasil_DT, df_brasil_RF, df_brasil_RL = [df_brasil.copy() for _ in range(4)]

# Para Peru
df_peru_RNN, df_peru_DT, df_peru_RF, df_peru_RL = [df_peru.copy() for _ in range(4)]

# Para Colombia/Ecuador
df_colombia_ecuador_RNN, df_colombia_ecuador_DT, df_colombia_ecuador_RF, df_colombia_ecuador_RL = [df_colombia_ecuador.copy() for _ in range(4)]


In [6]:
def split_data(df, test_size=0.2, random_state=42):
    return train_test_split(df, test_size=test_size, random_state=random_state, shuffle=False)

# Para Brasil
train_brasil_RNN, test_brasil_RNN = split_data(df_brasil_RNN)
train_brasil_DT, test_brasil_DT = split_data(df_brasil_DT)
train_brasil_RF, test_brasil_RF = split_data(df_brasil_RF)
train_brasil_RL, test_brasil_RL = split_data(df_brasil_RL)

# Para Peru
train_peru_RNN, test_peru_RNN = split_data(df_peru_RNN)
train_peru_DT, test_peru_DT = split_data(df_peru_DT)
train_peru_RF, test_peru_RF = split_data(df_peru_RF)
train_peru_RL, test_peru_RL = split_data(df_peru_RL)

# Para Colombia/Ecuador
train_colombia_ecuador_RNN, test_colombia_ecuador_RNN = split_data(df_colombia_ecuador_RNN)
train_colombia_ecuador_DT, test_colombia_ecuador_DT = split_data(df_colombia_ecuador_DT)
train_colombia_ecuador_RF, test_colombia_ecuador_RF = split_data(df_colombia_ecuador_RF)
train_colombia_ecuador_RL, test_colombia_ecuador_RL = split_data(df_colombia_ecuador_RL)

## Checando os Dados

Desenvolvemos esta função (*check_split*) e (*check_content*) para assegurar a correta separação dos DataFrames com os dados adequados, visando a obtenção de um modelo o mais preciso possível.

In [7]:
def check_split(train, test, original):
    print(f"Tamanho original: {original.shape}")
    print(f"Tamanho do treino: {train.shape}")
    print(f"Tamanho do teste: {test.shape}")
    print(f"Soma das partes: {train.shape[0] + test.shape[0]}")

# Brasil
print("Brasil:")
check_split(train_brasil_RNN, test_brasil_RNN, df_brasil_RNN)

# Peru
print("\nPeru:")
check_split(train_peru_RNN, test_peru_RNN, df_peru_RNN)

# Colombia/Ecuador
print("\nColombia/Ecuador:")
check_split(train_colombia_ecuador_RNN, test_colombia_ecuador_RNN, df_colombia_ecuador_RNN)

# Visualizando as primeiras e últimas linhas
def check_content(train, test):
    print("\nPrimeiras linhas do treino:")
    print(train.head())
    print("\nÚltimas linhas do treino:")
    print(train.tail())
    print("\nPrimeiras linhas do teste:")
    print(test.head())
    print("\nÚltimas linhas do teste:")
    print(test.tail())

print("\nVisualizando o conteúdo - Brasil RNN:")
check_content(train_brasil_RNN, test_brasil_RNN)
print("\nVisualizando o conteúdo - Peru RNN:")
check_content(train_peru_RNN, test_peru_RNN)
print("\nVisualizando o conteúdo - Colombia/Ecuador RNN:")
check_content(train_colombia_ecuador_RNN, test_colombia_ecuador_RNN)


Brasil:
Tamanho original: (23, 3)
Tamanho do treino: (18, 3)
Tamanho do teste: (5, 3)
Soma das partes: 23

Peru:
Tamanho original: (23, 3)
Tamanho do treino: (18, 3)
Tamanho do teste: (5, 3)
Soma das partes: 23

Colombia/Ecuador:
Tamanho original: (23, 3)
Tamanho do treino: (18, 3)
Tamanho do teste: (5, 3)
Soma das partes: 23

Visualizando o conteúdo - Brasil RNN:

Primeiras linhas do treino:
  Month/Year  Value Country
0 2022-02-01   0.55  Brasil
1 2022-03-04   0.69  Brasil
2 2022-04-04   0.70  Brasil
3 2022-05-05   0.70  Brasil
4 2022-06-05   0.66  Brasil

Últimas linhas do treino:
   Month/Year  Value Country
13 2023-03-04   0.71  Brasil
14 2023-04-04   0.67  Brasil
15 2023-05-05   0.70  Brasil
16 2023-06-05   0.70  Brasil
17 2023-07-06   0.71  Brasil

Primeiras linhas do teste:
   Month/Year  Value Country
18 2023-08-06   0.70  Brasil
19 2023-09-06   0.73  Brasil
20 2023-10-07   0.75  Brasil
21 2023-11-07   0.75  Brasil
22 2023-12-08   0.74  Brasil

Últimas linhas do teste:
   Mont

## Entendendo Root Mean Squared Error (RMSE)

O Root Mean Square Error (RMSE), ou Erro Quadrático Médio da Raiz, é uma métrica amplamente utilizada para avaliar a precisão de modelos de regressão. Ele mede a diferença entre os valores previstos por um modelo e os valores reais observados. Aqui está uma explicação mais detalhada:

### Definição e Cálculo

* O RMSE é calculado como a raiz quadrada da média dos quadrados dos erros.

### Interpretação

* Valores Próximos de Zero: Indicam um bom ajuste do modelo, significando que as previsões estão muito próximas dos valores reais.
* Valores Altos: Sugerem um mau ajuste, indicando que há uma grande discrepância entre os valores observados e preditos.

### Vantagens

* Facilidade de Interpretação, o RMSE é fácil de interpretar, pois está na mesma unidade dos dados observados.
* Penalização de Erros Grandes, penaliza mais os erros grandes do que os pequenos, o que pode ser útil em contextos onde erros grandes são mais problemáticos.

### Limitações

* Sensível a Outliers, o RMSE pode ser influenciado por valores extremos, o que pode distorcer a avaliação do ajuste do modelo.
* Dependência da Escala, o RMSE é dependente da escala dos dados, o que pode dificultar a comparação entre diferentes conjuntos de dados ou modelos.

### Aplicações

O RMSE é amplamente utilizado em diversas áreas, como:

* Previsão de Séries Temporais, ao avaliar a precisão de modelos que preveem valores futuros com base em dados históricos.
* Modelos de Regressão, para avaliar a precisão de modelos que relacionam uma variável dependente com uma ou mais variáveis independentes.
* Machine Learning, avaliar a precisão de modelos preditivos em tarefas de regressão


## Pré-processamento e criação e treinamento do modelo RNN.

In [8]:
def train_and_forecast_rnn(data, target_col, n_steps=3):

    # Escalar os dados
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data[target_col].values.reshape(-1, 1))

    # Dividir os dados em treino e teste
    train_size = int(len(data_scaled) * 0.8)
    train_scaled = data_scaled[:train_size]
    test_scaled = data_scaled[train_size:]

    # Criar listas vazias para armazenar os dados sequenciais
    X_train, y_train = [], []
    X_test, y_test = [], []

    # Gerar sequências de treino
    for i in range(n_steps, len(train_scaled)):
        X_train.append(train_scaled[i-n_steps:i, 0])
        y_train.append(train_scaled[i, 0])

    # Gerar sequências de teste
    for i in range(n_steps, len(test_scaled)):
        X_test.append(test_scaled[i-n_steps:i, 0])
        y_test.append(test_scaled[i, 0])

    # Converter as listas em arrays NumPy e redimensionar
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

    X_test, y_test = np.array(X_test), np.array(y_test)
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Criar o modelo RNN
    model_rnn = Sequential()
    model_rnn.add(SimpleRNN(50, activation='relu', input_shape=(n_steps, 1)))
    model_rnn.add(Dense(1))
    model_rnn.compile(optimizer='adam', loss='mse')

    # Treinar o modelo
    model_rnn.fit(X_train, y_train, epochs=200, verbose=0)

    # Previsão para o próximo mês (usando os últimos n_steps valores do conjunto de teste)
    X_input = test_scaled[-n_steps:]
    X_input = X_input.reshape((1, n_steps, 1))
    forecast_rnn = model_rnn.predict(X_input)
    forecast_rnn = scaler.inverse_transform(forecast_rnn)

    # Calcular o RMSE
    rmse = np.sqrt(mean_squared_error(y_test, model_rnn.predict(X_test)))

    print(f'Root Mean Squared Error (RMSE) RNN : {rmse}')
    print(f'Previsão RNN para o próximo mês: {forecast_rnn[0][0]:.4f}')

    return rmse, forecast_rnn[0][0]

# Exemplo de uso da função com os datasets dos países
print("Brasil:")
rmse_brasil, forecast_brasil = train_and_forecast_rnn(df_brasil_RNN, 'Value')

print("\nPeru:")
rmse_peru, forecast_peru = train_and_forecast_rnn(df_peru_RNN, 'Value')

print("\nColombia/Ecuador:")
rmse_colombia_ecuador, forecast_colombia_ecuador = train_and_forecast_rnn(df_colombia_ecuador_RNN, 'Value')


Brasil:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Root Mean Squared Error (RMSE): 0.22650091298619313
Previsão para o próximo mês: 0.6662

Peru:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
Root Mean Squared Error (RMSE): 0.1914679793324452
Previsão para o próximo mês: 0.7436

Colombia/Ecuador:


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Root Mean Squared Error (RMSE): 0.07567020403830879
Previsão para o próximo mês: 0.7582


O modelo para Colombia/Ecuador apresentou o menor RMSE (0.0757), indicando que, dentre os três países, esse modelo foi o mais preciso em capturar o padrão dos dados históricos e realizar previsões.

Peru também apresentou um bom desempenho, com um RMSE de 0.1915, sugerindo que o modelo para esse país está fazendo previsões relativamente precisas.
Brasil teve o maior RMSE (0.2265), o que indica que o modelo para este país é o menos preciso entre os três, embora ainda seja relativamente aceitável dependendo do contexto do problema.

## Os valores previstos para o próximo mês são:

* Brasil: 66.61%
* Peru: 74.36%
* Colombia/Ecuador: 75.82%

Esses valores representam a previsão do modelo para o valor (*Value*. Que consiste na porcentagem na qual o pais ira bater a meta) em janeiro de 2024.
Colombia/Ecuador tem a maior previsão (0.7582) ou 75.82% , seguido pelo Peru (0.7436) ou 74.36% e Brasil (0.6662) ou 66.61%.

Como as previsões estão próximas dos valores médios observados nos dados de treino, isso sugere que os modelos estão fazendo previsões que seguem as tendências gerais dos dados históricos.




---



## Modelo Regressão Linear.


In [9]:
def train_and_forecast_lr(data, target_col, date_col, forecast_date):

    # Criar novas features
    data['Mes'] = data[date_col].dt.month
    data['Ano'] = data[date_col].dt.year

    # Dividir os dados em treino e teste
    train_size = int(len(data) * 0.8)
    train_df = data[:train_size]
    test_df = data[train_size:]

    # Separar features e target
    X_train = train_df[['Mes', 'Ano']]
    y_train = train_df[target_col]
    X_test = test_df[['Mes', 'Ano']]
    y_test = test_df[target_col]

    # Criar e treinar o modelo
    model_lr = LinearRegression()
    model_lr.fit(X_train, y_train)

    # Fazer a previsão no conjunto de teste
    y_pred = model_lr.predict(X_test)

    # Calcular o RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Fazer a previsão para a data especificada
    forecast_df = pd.DataFrame({'Mes': [forecast_date.month], 'Ano': [forecast_date.year]})
    forecast_value = model_lr.predict(forecast_df)

    print(f'Root Mean Squared Error (RMSE) Regressão Linear: {rmse}')
    print(f'Previsão Regressão Linear para {forecast_date}: {forecast_value[0]:.4f}')

    return rmse, forecast_value[0]

# Usando a função para vermos os dados dos paises dispostos no nosso Forecast original.
forecast_date = pd.Timestamp('2024-01-01')

print("Brasil:")
rmse_brasil, forecast_brasil = train_and_forecast_lr(df_brasil_RL, 'Value', 'Month/Year', forecast_date)

print("\nPeru:")
rmse_peru, forecast_peru = train_and_forecast_lr(df_peru_RL, 'Value', 'Month/Year', forecast_date)

print("\nColombia/Ecuador:")
rmse_colombia_ecuador, forecast_colombia_ecuador = train_and_forecast_lr(df_colombia_ecuador_RL, 'Value', 'Month/Year', forecast_date)


Brasil:
Root Mean Squared Error (RMSE): 0.013737524194859671
Previsão para 2024-01-01 00:00:00: 0.7216

Peru:
Root Mean Squared Error (RMSE): 0.08972135149196127
Previsão para 2024-01-01 00:00:00: 0.7329

Colombia/Ecuador:
Root Mean Squared Error (RMSE): 0.03424980428577501
Previsão para 2024-01-01 00:00:00: 0.7652




---



O RMSE mede a diferença entre os valores preditos e os valores reais observados. Quanto menor o RMSE, mais preciso é o modelo.

* Brasil: RMSE = 0.0137
* Peru: RMSE = 0.0897
* Colombia/Ecuador: RMSE = 0.0342

O modelo para o Brasil apresenta o menor RMSE (0.0137), indicando alta precisão na previsão de se o país atingirá a meta ou não.

Colombia/Ecuador tem um RMSE moderadamente baixo (0.0342), sugerindo que o modelo é bastante preciso, mas não tanto quanto o do Brasil.

Peru apresenta o maior RMSE (0.0897), o que sugere que o modelo para esse país tem uma maior margem de erro e pode ser menos confiável em comparação aos outros dois.


Essas previsões representam a porcentagem esperada de metas atingidas em janeiro de 2024.

* Brasil: 72.16%
* Peru: 73.29%
* Colombia/Ecuador: 76.52%

## Modelo Decision Tree


In [10]:
def train_and_forecast_dt(data, target_col, date_col, forecast_date):

    # Converter a coluna 'Month/Year' para uma representação numérica
    data[date_col] = data[date_col].apply(lambda x: x.toordinal())

    # Dividir os dados em treino e teste
    train_size = int(len(data) * 0.8)
    train_df = data[:train_size]
    test_df = data[train_size:]

    # Preparar os dados
    X_train = train_df[[date_col]]
    y_train = train_df[target_col]
    X_test = test_df[[date_col]]
    y_test = test_df[target_col]

    # Criar e treinar o modelo
    model_dt = DecisionTreeRegressor(random_state=42)
    model_dt.fit(X_train, y_train)

    # Fazer a previsão no conjunto de teste
    y_pred = model_dt.predict(X_test)

    # Calcular o RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Fazer a previsão para a data especificada
    forecast_df = pd.DataFrame({date_col: [forecast_date.toordinal()]})
    forecast_value = model_dt.predict(forecast_df)

    print(f'Root Mean Squared Error (RMSE) Decision Tree: {rmse}')
    print(f'Previsão Decision Tree para {forecast_date}: {forecast_value[0]:.4f}')

    return rmse, forecast_value[0]

# uso da função com os datasets dos países
forecast_date = pd.Timestamp('2024-01-01')

print("Brasil:")
rmse_brasil, forecast_brasil = train_and_forecast_dt(df_brasil_DT, 'Value', 'Month/Year', forecast_date)

print("\nPeru:")
rmse_peru, forecast_peru = train_and_forecast_dt(df_peru_DT, 'Value', 'Month/Year', forecast_date)

print("\nColombia/Ecuador:")
rmse_colombia_ecuador, forecast_colombia_ecuador = train_and_forecast_dt(df_colombia_ecuador_DT, 'Value', 'Month/Year', forecast_date)


Brasil:
Root Mean Squared Error (RMSE): 0.03033150177620623
Previsão para 2024-01-01 00:00:00: 0.7100

Peru:
Root Mean Squared Error (RMSE): 0.10459445491994303
Previsão para 2024-01-01 00:00:00: 0.6700

Colombia/Ecuador:
Root Mean Squared Error (RMSE): 0.02144761058952724
Previsão para 2024-01-01 00:00:00: 0.7400




---



O RMSE, que mede a precisão do modelo, continua sendo um indicador fundamental para avaliar a confiabilidade das previsões.

* Brasil: RMSE = 0.0303
* Peru: RMSE = 0.1046
* Colombia/Ecuador: RMSE = 0.0214


Colombia/Ecuador apresenta o menor RMSE (0.0214), indicando que o modelo para esse grupo de países é o mais preciso.

Brasil também mostra um RMSE relativamente baixo (0.0303), sugerindo alta precisão na previsão para esse país, embora não tão alta quanto para Colombia/Ecuador.

Peru tem o maior RMSE (0.1046), o que indica que o modelo para esse país é o menos preciso e, portanto, as previsões são menos confiáveis.


As previsões indicam a porcentagem esperada de metas atingidas em janeiro de 2024.

* Brasil: 71.00%
* Peru: 67.00%
* Colombia/Ecuador: 74.00%



## Modelo Random Forest



In [12]:
def train_and_forecast_rf(data, target_col, date_col, forecast_date):

    # Separar as features (X) e a target (y)
    X = data[[date_col]]
    y = data[target_col]

    # Dividir os dados em treino e teste
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Criar e treinar o modelo Random Forest
    model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    model_rf.fit(X_train, y_train)

    # Fazer previsão no conjunto de teste
    y_pred = model_rf.predict(X_test)

    # Calcular o RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Fazer previsão para a data especificada
    forecast_df = pd.DataFrame({date_col: [forecast_date]})
    forecast_value = model_rf.predict(forecast_df)

    print(f'Root Mean Squared Error (RMSE) Random Forest: {rmse}')
    print(f'Previsão Random Forest para {forecast_date}: {forecast_value[0]}')

    return rmse, forecast_value[0]

# uso da função com os datasets dos países
forecast_date = pd.Timestamp('2024-01-01')

print("Brasil:")
rmse_brasil, forecast_brasil = train_and_forecast_rf(df_brasil_RF, 'Value', 'Month/Year', forecast_date)

print("\nPeru:")
rmse_peru, forecast_peru = train_and_forecast_rf(df_peru_RF, 'Value', 'Month/Year', forecast_date)

print("\nColombia/Ecuador:")
rmse_colombia_ecuador, forecast_colombia_ecuador = train_and_forecast_rf(df_colombia_ecuador_RF, 'Value', 'Month/Year', forecast_date)


Brasil:
Root Mean Squared Error (RMSE) Random Forest: 0.033336016558671064
Previsão Random Forest para 2024-01-01 00:00:00: 0.7063000000000001

Peru:
Root Mean Squared Error (RMSE) Random Forest: 0.12073922312156801
Previsão Random Forest para 2024-01-01 00:00:00: 0.6494000000000011

Colombia/Ecuador:
Root Mean Squared Error (RMSE) Random Forest: 0.019241621553289085
Previsão Random Forest para 2024-01-01 00:00:00: 0.7268000000000002




---



O RMSE é um indicador da precisão do modelo, com valores mais baixos indicando previsões mais precisas.

* Brasil: RMSE = 0.0333
* Peru: RMSE = 0.1207
* Colombia/Ecuador: RMSE = 0.0192


Colombia/Ecuador continua a ter o menor RMSE (0.0192), indicando que o modelo Random Forest é altamente preciso para esse grupo de países.

Brasil tem um RMSE moderadamente baixo (0.0333), sugerindo que o modelo é preciso, mas com uma leve incerteza em comparação com Colombia/Ecuador.

Peru apresenta o maior RMSE (0.1207), indicando que as previsões para esse país são as menos precisas entre os três, sugerindo uma maior incerteza na previsão.


As previsões refletem a porcentagem esperada de metas atingidas em janeiro de 2024.

* Brasil: 70.63%
* Peru: 64.94%
* Colombia/Ecuador: 72.68%

## Conclusões Finais

Para decidir qual modelo deve ser utilizado na produção com base nos RMSEs apresentados para os três países (Brasil, Peru e Colombia/Ecuador), vamos analisar e comparar o desempenho de cada modelo.

1. Modelo RNN
* Brasil: RMSE = 0.2265
* Peru: RMSE = 0.1915
* Colombia/Ecuador: RMSE = 0.0757


O modelo RNN apresenta o maior RMSE entre os modelos, especialmente para o Brasil e Peru. Isso indica que este modelo é menos preciso e, portanto, menos adequado para ser implementado em produção.

2. Modelo de Regressão Linear
* Brasil: RMSE = 0.0137
* Peru: RMSE = 0.0897
* Colombia/Ecuador: RMSE = 0.0342


A Regressão Linear apresenta o menor RMSE para o Brasil, o que sugere que é o modelo mais preciso para esse país.
Para Peru e Colombia/Ecuador, os RMSEs também são baixos, especialmente em comparação com o RNN e Decision Tree.
A Regressão Linear oferece um bom equilíbrio entre os países, com um excelente desempenho no Brasil.

3. Modelo Decision Tree
* Brasil: RMSE = 0.0303
* Peru: RMSE = 0.1046
* Colombia/Ecuador: RMSE = 0.0214


A Decision Tree apresenta um desempenho bom para Colombia/Ecuador, mas é menos precisa para o Brasil e Peru em comparação com a Regressão Linear.
Embora seja competitiva, especialmente para Colombia/Ecuador, a Regressão Linear ainda supera a Decision Tree no geral.

4. Modelo Random Forest
* Brasil: RMSE = 0.0333
* Peru: RMSE = 0.1207
* Colombia/Ecuador: RMSE = 0.0192


O Random Forest apresenta o menor RMSE para Colombia/Ecuador, indicando uma alta precisão para este país.
No entanto, para o Brasil e Peru, o Random Forest é menos preciso em comparação com a Regressão Linear.
Este modelo é excelente para Colombia/Ecuador, mas menos competitivo para os outros países.

##**Conclusão**:

Regressão Linear emerge como o modelo mais equilibrado e preciso entre os três países, especialmente destacando-se para o Brasil com o menor RMSE (0.0137).
Random Forest é altamente preciso para Colombia/Ecuador, mas a Regressão Linear ainda apresenta um bom desempenho geral para todos os países, com o menor RMSE para o Brasil e resultados competitivos para os demais.

## **Decisão**:

O modelo de Regressão Linear é o mais indicado para ser utilizado em produção, pois oferece a melhor precisão geral para os três países, garantindo previsões mais consistentes e confiáveis em diferentes cenários.